<a href="https://colab.research.google.com/github/SBoo9/TRANSFORMERS_AND_BERT/blob/main/BERT_TEST_MODULE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text

ModuleNotFoundError: No module named 'tensorflow_text'

In [ ]:
preprocessor_url="https://kaggle.com/models/tensorflow/albert/TensorFlow2/en-preprocess/3"
encoder_url="https://www.kaggle.com/models/tensorflow/albert/TensorFlow2/en-base/3"

In [ ]:
bert_preprocess_model=hub.KerasLayer(preprocessor_url)

In [ ]:
text_text=['Ratatouie was nice movie','I love fears of fathom']
text_preprocessed=bert_preprocess_model(text_text)
text_preprocessed.keys()

In [ ]:
text_preprocessed['input_mask']

In [ ]:
bert_model=hub.KerasLayer(encoder_url)
bert_result=bert_model(text_preprocessed)
bert_result.keys()

In [ ]:
bert_result['pooled_output']

In [ ]:
##!pip install tensorflow_text

In [ ]:
from transformers import BertModel,BertConfig

In [ ]:
configuration=BertConfig()
configuration

In [ ]:
model=BertModel(configuration)

In [ ]:
configuration=model.config

In [ ]:
configuration

In [ ]:
from transformers import AutoTokenizer, TFBertModel
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = TFBertModel.from_pretrained("google-bert/bert-base-uncased")

inputs = tokenizer("Hello, my dog is cute", return_tensors="tf")
outputs = model(inputs)

last_hidden_states = outputs.last_hidden_state

In [ ]:
outputs

In [ ]:
last_hidden_states

In [ ]:
from transformers import AutoTokenizer, TFBertForMaskedLM
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = TFBertForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

inputs = tokenizer("The capital of France is [MASK].", return_tensors="tf")
logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = tf.where((inputs.input_ids == tokenizer.mask_token_id)[0])
selected_logits = tf.gather_nd(logits[0], indices=mask_token_index)

predicted_token_id = tf.math.argmax(selected_logits, axis=-1)
tokenizer.decode(predicted_token_id)

In [ ]:
labels = tokenizer("The capital of France is Paris.", return_tensors="tf")["input_ids"]
# mask labels of non-[MASK] tokens
labels = tf.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

outputs = model(**inputs, labels=labels)
round(float(outputs.loss), 2)

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFBertForNextSentencePrediction

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = TFBertForNextSentencePrediction.from_pretrained("google-bert/bert-base-uncased")

prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."
encoding = tokenizer(prompt, next_sentence, return_tensors="tf")

logits = model(encoding["input_ids"], token_type_ids=encoding["token_type_ids"])[0]
assert logits[0][0] < logits[0][1]  # the next sentence was random

In [ ]:
from transformers import AutoTokenizer, TFBertForSequenceClassification
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("ydshieh/bert-base-uncased-yelp-polarity")
model = TFBertForSequenceClassification.from_pretrained("ydshieh/bert-base-uncased-yelp-polarity")

inputs = tokenizer("Hello, my dog is cute", return_tensors="tf")

logits = model(**inputs).logits

predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]

In [4]:
from transformers import pipeline
claddifier=pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [5]:
classifier=claddifier

In [11]:
classifier("Ur father was a dog, but a great one")

[{'label': 'POSITIVE', 'score': 0.9959536790847778}]

In [12]:
classifier=pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")

config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


In [15]:
print(classifier("जी नहीं, किरण दूध नहीं पीती है"))

[{'label': 'Very Negative', 'score': 0.5678442716598511}]
